In [1]:
import pandas as pd
from pathlib import Path
import uuid

In [2]:
fincausal_task1_train_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/task1/train_on_trial_and_test_on_practice/train.csv')
causaly_all_data_path = Path('/media/sarthak/HDD/data_science/fnp_resources/data/extra/causaly.csv')

In [3]:
causaly_df = pd.read_csv(causaly_all_data_path)
causaly_df.head()

,sentence,causal
0,"Results indicated that boys with ADHD , relati...",1
1,Ticagrelor appears to provide higher value for...,0
2,"Whatever the mechanism , active smoking is an ...",1
3,"In this study , we examined whether use of an ...",0
4,"Using causal inference testing , we searched c...",0


In [4]:
causaly_df.shape

(2000, 2)

In [5]:
fincausal_task1_train_df = pd.read_csv(fincausal_task1_train_path)
fincausal_task1_train_df.head()

,Index,Text,Gold,unique_id
0,115.00001,Seeking Alpha ETF Analysis | About: Global...,0,59aabda8-b867-43eb-bb6d-c9ba59fb8025
1,330.00001,Options for clients concerned about how IHT l...,0,90866e7b-1440-4d65-a333-17389c1a3705
2,315.00016,Cackley said that figure indicates loan servi...,0,dbbd03a4-2b62-4f99-bc0b-597a64f2ce60
3,184.00007,A taxpayer has normally been required to file...,0,72440cd6-7729-4337-b476-3d0b29eec078
4,241.00002,I was excited to listen…and thanks to Paula f...,0,ed17a935-0095-4b3b-99f6-9798c32d5b68


In [6]:
fincausal_task1_train_df.shape

(6902, 4)

In [7]:
# what's the distribution of label?
causal_distribution = fincausal_task1_train_df.Gold.value_counts() / len(fincausal_task1_train_df)
causal_distribution

0    0.933642
1    0.066358
Name: Gold, dtype: float64

In [8]:
# how many causal can I take from causaly?
causaly_noncausal = causaly_df.loc[causaly_df.causal == 0]
causaly_causal = causaly_df.loc[causaly_df.causal == 1]

causaly_causal_resampled = causaly_causal.sample(n=int((causal_distribution[1]*100*len(causaly_noncausal))/(100-(causal_distribution[1]*100))))

causaly_resampled = pd.concat([causaly_noncausal, causaly_causal_resampled])
causaly_resampled.causal.value_counts() / len(causaly_resampled)

0    0.933684
1    0.066316
Name: causal, dtype: float64

In [9]:
# combine both the datasets
causly_resampled_formatted_df = pd.DataFrame({
    'Index': None,
    'Text': causaly_resampled.sentence,
    'Gold': causaly_resampled.causal,
    'unique_id': [str(uuid.uuid4()) for _ in range(len(causaly_resampled))]
})
causly_resampled_formatted_df.head()

,Index,Text,Gold,unique_id
1,None,Ticagrelor appears to provide higher value for...,0,1f3e7605-75d0-4442-919d-6ec833fe8af1
3,None,"In this study , we examined whether use of an ...",0,d3588b63-a5ab-42db-8504-fe799eaabfb7
4,None,"Using causal inference testing , we searched c...",0,a23b69e9-192c-4cee-a49c-b00c9de63972
6,None,Data were collected on the incidence of POAF l...,0,e55c346c-b931-4517-a8eb-d14cf07302f9
8,None,"In the current study , women were losing BMC ,...",0,f77cd3a5-69cb-41ba-9f25-9b3eea38ce8d


In [10]:
causly_resampled_formatted_df.Gold.value_counts() / len(causly_resampled_formatted_df)

0    0.933684
1    0.066316
Name: Gold, dtype: float64

In [11]:
fincausal_task1_train_combined_causaly = pd.concat([fincausal_task1_train_df, causly_resampled_formatted_df])
fincausal_task1_train_combined_causaly.shape

(7852, 4)

In [12]:
fincausal_task1_train_combined_causaly.Gold.value_counts() / len(fincausal_task1_train_combined_causaly)

0    0.933647
1    0.066353
Name: Gold, dtype: float64

In [13]:
fincausal_task1_train_combined_causaly.to_csv('/media/sarthak/HDD/data_science/fnp_resources/data/task1/train_on_trial_and_test_on_practice/train_with_causaly_balanced.csv', index=False)